In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [9]:
import os
cwd = os.getcwd()
print(cwd)

/Users/jessica/Desktop/CSE8803IUC-main


In [49]:
train_lda = pd.read_csv('/Users/jessica/Desktop/CSE8803IUC-main/data/b_training_lda_final.csv', index_col=['date'])
test_lda = pd.read_csv('/Users/jessica/Desktop/CSE8803IUC-main/data/b_testing_lda_final.csv', index_col=['date'])
nlp_data = pd.read_csv('/Users/jessica/Desktop/CSE8803IUC-main/data/Boston_feat.csv', index_col=['date'])

In [50]:
X_train = train_lda.join(nlp_data)
X_test = test_lda.join(nlp_data)

In [51]:
X_train.shape, X_test.shape

((292, 19), (73, 19))

In [52]:
gt = pd.read_csv('/Users/jessica/Desktop/CSE8803IUC-main/data/Boston22_groundtruth.csv')

In [53]:
gt['Total']= gt.iloc[:, 1:].sum(axis=1)
gt['date'] = gt['OCCURRED_ON_DATE'].str[:10]

In [54]:
gt = gt.set_index('date')

In [55]:
Y_train = gt.reindex(X_train.index)['Total']
Y_test = gt.reindex(X_test.index)['Total']

In [56]:
# generate data for n-th iteration for k-fold cross validation
def getdata_cv(Xtrain, Ytrain, k=5, n=0):
    batch_size = int(Xtrain.shape[0] / k)
    if n == 0:
        Xtrain_batch = Xtrain[batch_size:, :]
        Ytrain_batch = Ytrain[batch_size:]
        Xtest_batch = Xtrain[:batch_size, :]
        Ytest_batch = Ytrain[:batch_size]
    elif n == k-1:
        Xtrain_batch = Xtrain[:n*batch_size, :]
        Ytrain_batch = Ytrain[:n*batch_size]
        Xtest_batch = Xtrain[n*batch_size:, :]
        Ytest_batch = Ytrain[n*batch_size:]
    else:
        start_idx = n*batch_size
        end_idx = (n+1)*batch_size
        Xtrain_batch = np.concatenate((Xtrain[:start_idx, :], Xtrain[end_idx:, :]))
        Ytrain_batch = np.concatenate((Ytrain[:start_idx], Ytrain[end_idx:]))
        Xtest_batch = Xtrain[start_idx:end_idx, :]
        Ytest_batch = Ytrain[start_idx:end_idx]
    return Xtrain_batch, Ytrain_batch, Xtest_batch, Ytest_batch


In [71]:
# hyper parameter
percentage = [0.05, 0.1, 0.2]

In [72]:
def accuracy(pred, gt, percentage=0.05):
    gt_upper = gt + gt * percentage
    gt_lower = gt - gt * percentage
    return np.mean((pred <= gt_upper) & (pred >= gt_lower))

In [77]:
from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB()
#clf.fit(X, Y)
def train_NB(Xtrain, Ytrain, percentage, k=5):
    loss_sum = 0
    acc_sum = 0
    for i in range(k):
        Xtrain_batch, Ytrain_batch, Xtest_batch, Ytest_batch = getdata_cv(Xtrain, Ytrain, k=k, n=i)
        model = GaussianNB().fit(Xtrain_batch, Ytrain_batch)
        pred = model.predict(Xtest_batch)
        loss_sum += mean_squared_error(pred, Ytest_batch)
        acc_sum += accuracy(pred, Ytest_batch, percentage)
        print(model.score(Xtest_batch, Ytest_batch))
        #score: Mean accuracy of self.predict(X) w.r.t. y.
    return loss_sum/k, acc_sum/k

In [78]:
result_loss = np.zeros(len(percentage))
result_acc = np.zeros(len(percentage))
for j in range(len(percentage)):
    loss, acc = train_NB(X_train.to_numpy(), Y_train.to_numpy(), percentage[j])
    result_loss[j] = loss
    result_acc[j] = acc


0.034482758620689655
0.0
0.034482758620689655
0.0
0.0
0.034482758620689655
0.0
0.034482758620689655
0.0
0.0
0.034482758620689655
0.0
0.034482758620689655
0.0
0.0


In [79]:
result_loss

array([4921.02896552, 4921.02896552, 4921.02896552])

In [80]:
result_acc

array([0.17804598, 0.42735632, 0.74609195])